In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import Lambda, Flatten, Dense
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.layers import Layer
from tensorflow.keras import backend as K
import cv2 
K.set_image_data_format('channels_last')
import os
import numpy as np
from numpy import genfromtxt
import tensorflow as tf
import PIL

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [5]:
print(tf.__version__)

2.4.1


In [6]:
from tensorflow.keras.models import model_from_json

json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
model.load_weights('weights.h5')

2024-02-11 09:19:27.900496: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2024-02-11 09:19:27.900746: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
print(model.inputs)
print(model.outputs)

[<KerasTensor: shape=(None, 160, 160, 3) dtype=float32 (created by layer 'input_1')>]
[<KerasTensor: shape=(None, 128) dtype=float32 (created by layer 'Bottleneck_BatchNorm')>]


In [8]:
model.summary()

Model: "inception_resnet_v1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 160, 160, 3) 0                                            
__________________________________________________________________________________________________
Conv2d_1a_3x3 (Conv2D)          (None, 79, 79, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
Conv2d_1a_3x3_BatchNorm (BatchN (None, 79, 79, 32)   96          Conv2d_1a_3x3[0][0]              
__________________________________________________________________________________________________
Conv2d_1a_3x3_Activation (Activ (None, 79, 79, 32)   0           Conv2d_1a_3x3_BatchNorm[0][0]    
________________________________________________________________________________

In [25]:
FRmodel = model

In [63]:
def img_to_encoding(image_path, model):
    # Load the image
    img = cv2.imread(image_path)
    
    # Convert the image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Load the pre-trained face detection cascade
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    
    # Detect faces in the image
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    
    # If no faces are detected, return None
    if len(faces) == 0:
        return None
    
    # Assuming only one face is detected, extract the first face
    (x, y, w, h) = faces[0]
    
    # Extract the face region from the image
    face_img = img[y:y+h, x:x+w]
    
    # Resize the face image to the required input size of the model
    face_img = cv2.resize(face_img, (160, 160))
    
    # Preprocess the image
    face_img = np.around(face_img / 255.0, decimals=12)
    
    # Expand the dimensions to match the input shape expected by the model
    x_train = np.expand_dims(face_img, axis=0)
    
    # Compute the embedding using the provided model
    embedding = model.predict_on_batch(x_train)
    
    # Normalize the embedding
    normalized_embedding = embedding / np.linalg.norm(embedding, ord=2)
    
    return normalized_embedding


In [72]:
database = {}
database["Ethan Hickey"] = img_to_encoding("Ethan Hickey.png", FRmodel)
database["Nafis Ishrak"] = img_to_encoding("Nafis Ishrak.png", FRmodel)
database["Brennan Graham"] = img_to_encoding("Brennan Graham.png", FRmodel)
database["Rodrigo Angulo"] = img_to_encoding("Rodrigo Angulo.png", FRmodel)
#database["Fahim Nafis"] = img_to_encoding("Fahim Nafis.jpeg", FRmodel)

In [73]:
for name, encoding in database.items():
    print("Name:", name)
    print("Encoding:", encoding)
    print()


Name: Ethan Hickey
Encoding: [[ 3.90026085e-02  2.49583144e-02 -1.13383017e-01  6.98103160e-02
   8.16577449e-02 -1.73872128e-01 -1.26197077e-02 -1.00897409e-01
  -2.18416769e-02 -4.85924967e-02 -1.61230117e-01  1.07523702e-01
   1.54702991e-01 -5.34793772e-02  4.75807302e-02 -1.38701588e-01
   8.57293978e-02 -4.09072498e-03  8.36821869e-02  4.34206463e-02
   9.19103995e-03 -4.72248606e-02 -7.22191036e-02 -8.29378963e-02
   1.02540687e-01  1.87084917e-02  3.30319963e-02  2.62153428e-02
   7.84755964e-03  1.09388739e-01  5.85859381e-02  2.04089656e-02
  -8.03171694e-02 -5.06250523e-02 -1.42564476e-01  1.35816913e-03
  -1.61583889e-02  3.11239548e-02 -6.81176558e-02 -1.16446536e-04
  -8.51251930e-03  1.63882732e-01 -7.35034719e-02  9.36702415e-02
  -3.87463234e-02  1.07487570e-02  6.00670353e-02  3.36518697e-02
  -2.68445481e-02  8.19761977e-02 -1.71149820e-01  8.70175511e-02
  -2.51786448e-02  1.30358666e-01  4.18196321e-02 -1.70539752e-01
  -6.25272393e-02  6.72150776e-02  1.56754609e-

In [80]:
def who_is_it(image_path, database, model):

    encoding = img_to_encoding(image_path, model)
    

    if encoding is None:
        print("No face detected in the image.")
        return None, None
    
 
    min_dist = 100
    identity = None
    for name, db_enc in database.items():
        dist = np.linalg.norm(encoding - db_enc)
        if dist < min_dist:
            min_dist = dist
            identity = name
    

    if min_dist > 0.7:
        print("Not in the database.")
        return min_dist, None
    

    print("It's " + str(identity) + ", the distance is " + str(min_dist))
    return min_dist, identity


In [ ]:
who_is_it("path to image", database, FRmodel)